In [10]:
import argparse
from SoccerNet.utils import getListGames
import os
import json
from tqdm import tqdm
import numpy as np
from SoccerNet.Evaluation.ActionSpotting import evaluate

In [25]:
parser = argparse.ArgumentParser()
# parser.add_argument("--anno_file",default='')
parser.add_argument("--source_test_path",default='/youtu_pedestrian_detection/chengzhilin/program/soccernet/models/MVIT_FULL/outputs_test')#输入test文件夹 
parser.add_argument("--output_test_path",default='/youtu_pedestrian_detection/zhuhe/soccernet/submit/output')  #筛选后test文件夹

parser.add_argument("--source_challenge_path",default='/youtu_pedestrian_detection/chengzhilin/program/soccernet/models/MVIT_FULL/outputs_challenge')#输入challenge文件夹 
parser.add_argument("--output_challenge_path", default="/youtu_pedestrian_detection/zhuhe/soccernet/submit/challenge_add_0.4")  #输出challenge文件夹
parser.add_argument('--time_start', type=int,   default=0,     help='Size of the chunk (in seconds)' )
parser.add_argument('--time_end', type=int,   default=1,     help='Size of the chunk (in seconds)' )
parser.add_argument('--time_gap',  type=int,   default=0.1,     help='Size of the chunk (in seconds)' )

parser.add_argument('--window_size',  type=int,   default=1,     help='Size of the chunk (in seconds)' )

args=parser.parse_args([])

In [12]:
os.makedirs(args.output_challenge_path,exist_ok='True')

In [13]:
def process_window(window):
    action_dic={}
    res=[]
    for pre in window:
              
        if pre['label'] not in action_dic:
            action_dic[pre['label']]=pre
        else:
            if float(pre['confidence']) > float(action_dic[pre['label']]['confidence']):
                action_dic[pre['label']]=pre
                
                
    for label in action_dic.keys():
        time=int(float(action_dic[label]['position']))
        time+=add_time*1000
        if time<0: time=0
        action_dic[label]['position']=str(int(time))
        
        res.append(action_dic[label])
    return res

In [14]:
total_results=[]
window_size=args.window_size

add_time=args.time_start
while add_time<=args.time_end:


    print(add_time)

    for cata in tqdm(getListGames(split="test")):
        

        file=os.path.join(args.source_test_path,cata,'results_spotting.json')
        with open(file,'r') as f:
            data=json.load(f)
        pres=data['predictions']
        pres.sort(key=lambda x:(int(x['half']),int(x['position'])))

        json_data = {} 
        total_pre={}
        json_data["UrlLocal"]=data['UrlLocal']
        json_data["predictions"]=[]

        last_half=1
        start,end=0,window_size
        window=[]
        for cur_pre,pre in enumerate(pres):


            if start<= int(pre['position'])/1000.0 <end:
                window.append(pre)
            else:
                json_data["predictions"]+=process_window(window)
                window=[pre]
                # while end<int(pre['position']):
                start+=window_size
                end+=window_size

                if last_half==1 and int(pre['half'])==2:
                    last_half=2
                    start,end=0,window_size

        json_data["predictions"]+=process_window(window)

        os.makedirs(os.path.join(args.output_test_path,cata),exist_ok='True')
        with open(os.path.join(args.output_test_path,cata,'results_spotting.json'),'w') as f:
            json.dump(json_data,f,indent=2)
    
    PATH_DATASET='/youtu_pedestrian_detection/junweil/action_datasets/soccernet'
    PATH_PREDICTIONS=args.output_test_path


    from SoccerNet.Evaluation.ActionSpotting import evaluate
    results = evaluate(SoccerNet_path=PATH_DATASET, Predictions_path=PATH_PREDICTIONS,
                       split="test", version=2, prediction_file="results_spotting.json", metric="tight")

    a_map,a_mAP_visible,a_mAP_unshown=results["a_mAP"],results["a_mAP_visible"],results["a_mAP_unshown"]
    total_results.append([add_time,a_map,a_mAP_visible,a_mAP_unshown])
    
    
    add_time+=args.time_gap

0


100%|██████████| 5/5 [00:14<00:00,  2.83s/it]


0.1


100%|██████████| 5/5 [00:13<00:00,  2.76s/it]


0.2


...

100%|██████████| 5/5 [00:13<00:00,  2.65s/it]


0.8999999999999999


100%|██████████| 5/5 [00:13<00:00,  2.65s/it]


0.9999999999999999


100%|██████████| 5/5 [00:13<00:00,  2.61s/it]


In [14]:
total_results

[[26.5, 0.3535285850986709, 0.43257104136907215, 0.12459536427751249]]

In [11]:
total_results

[[26.5, 0.37189007693837783, 0.45514818894529907, 0.13240919371158277]]

In [15]:
total_results.sort(key=lambda x:-x[1])
window_size=total_results[0][0]

In [16]:
total_results

[[0.4, 0.5500334271598455, 0.6277773366098063, 0.2988011337024556],
 [0.5, 0.5490917122506327, 0.6280780577037764, 0.29400246049331125],
 [0.6, 0.5472310532455537, 0.6269390831012711, 0.28892321982364844],
 [0.30000000000000004,
  0.5467036449930434,
  0.6220349210306454,
  0.30048090439486014],
 [0.7, 0.5442041592001741, 0.62319374285478, 0.28432045769303643],
 [0.2, 0.5400769653273818, 0.6144542857286631, 0.3011417840590246],
 [0.7999999999999999,
  0.5369053950043977,
  0.6167656389331209,
  0.2802232990352241],
 [0.1, 0.5324807066431241, 0.6051081932546492, 0.302009711647571],
 [0.8999999999999999,
  0.5280811957592768,
  0.6076353228090435,
  0.2749819006261527],
 [0, 0.5231123235011503, 0.5940917375828612, 0.30209996583449655],
 [0.9999999999999999,
  0.5145180448554506,
  0.5931507004330768,
  0.25619773143808644]]

In [112]:
window_size

10.0

In [113]:
total_results

[[10.0, 0.3635886168702596, 0.4452361951226681, 0.13416065650464098],
 [9.0, 0.3628183949734276, 0.4408861857435958, 0.13129837213006507],
 [8.0, 0.3623251507985085, 0.4394945123104102, 0.13513755799699895],
 [9.5, 0.36051810512867954, 0.43839204361662265, 0.13315908195648835],
 [8.5, 0.3597629987587144, 0.440094394295939, 0.13528863884583778],
 [7.0, 0.3574157910817314, 0.4357392284301309, 0.13575448732562678],
 [6.5, 0.3571480587524357, 0.43471564446898003, 0.13501414585936694],
 [7.5, 0.35633529300113514, 0.43441008626314975, 0.13633516850227909],
 [5.5, 0.35045527138076066, 0.4247491026570801, 0.13260608574209778],
 [6.0, 0.3503767726499682, 0.42360994858021006, 0.1333123736889443],
 [5.0, 0.3486040402767273, 0.4253135064499454, 0.1337106279009972],
 [4.0, 0.34435912502347854, 0.41643962807336105, 0.1324311198498365],
 [4.5, 0.3424705960086811, 0.41638542665409883, 0.1315793889648043],
 [3.5, 0.3370436435125208, 0.40952792182902686, 0.13061871362556607],
 [3.0, 0.32788232376722665,

In [28]:
add_time=0.4

In [29]:
add_time

0.4

In [26]:
window_size

0.4

In [30]:
for cata in tqdm(getListGames(split="challenge")):
    
    file=os.path.join(args.source_challenge_path,cata,'results_spotting.json')
    with open(file,'r') as f:
        data=json.load(f)
    pres=data['predictions']
    pres.sort(key=lambda x:(int(x['half']),int(x['position'])))
    
    json_data = {} 
    total_pre={}
    json_data["UrlLocal"]=data['UrlLocal']
    json_data["predictions"]=[]
    
    last_half=1
    start,end=0,window_size
    window=[]
    for cur_pre,pre in enumerate(pres):

        
        if start<= int(pre['position'])/1000.0 <end:
            window.append(pre)
        else:
            json_data["predictions"]+=process_window(window)
            window=[pre]
            start+=window_size
            end+=window_size
            
            if last_half==1 and int(pre['half'])==2:
                last_half=2
                start,end=0,window_size
         
    json_data["predictions"]+=process_window(window)
    
    os.makedirs(os.path.join(args.output_challenge_path,cata),exist_ok='True')
    with open(os.path.join(args.output_challenge_path,cata,'results_spotting.json'),'w') as f:
        json.dump(json_data,f,indent=2)
    
    


100%|██████████| 50/50 [00:16<00:00,  3.09it/s]
